## Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from konlpy.tag import Okt

## Data Load

In [4]:
Jungrang_reviews_sentiment = pd.read_excel('C:/Users/user/Desktop/saltlux_project/data/review_file_junrang.xlsx')
Jungrang_reviews_sentiment.head(2)

,name,content,sentiment
0,365mc모인이비인후과의원,사가정 모인 이비인후과 항상 만원이다,0
1,365mc모인이비인후과의원,좋아요,1


## Preprocess

In [5]:
Jungrang_reviews_sentiment['content'] = Jungrang_reviews_sentiment['content'].str.replace('\n', ' ')
Jungrang_reviews_sentiment['content'] = Jungrang_reviews_sentiment['content'].str.replace('[~!]', ' ', regex=True) # regex=True : 정규표현식으로 해석(디폴트)
Jungrang_reviews_sentiment['content'] = Jungrang_reviews_sentiment['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백 제외하고 모두 제거
Jungrang_reviews_sentiment['content'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
print(Jungrang_reviews_sentiment.isnull().sum())

name         0
content      7
sentiment    0
dtype: int64


In [6]:
Jungrang_sentiment = Jungrang_reviews_sentiment.dropna(how='any') # Null 값 제거
print('전처리 후 샘플의 개수 :',len(Jungrang_sentiment))

전처리 후 샘플의 개수 : 5966


## Tokenizer - 수정 필요

- 불용어 사전을 별도로 만들 필요가 있음

In [7]:
tokenizer = Okt()

In [8]:
stop_words = ['은', '는', '이', '가', '을', '를', '다', '의', '가', '이', '은', '한', '에', '하', '고', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
Jungrang_sentiment['tokenized'] = Jungrang_sentiment['content'].apply(tokenizer.morphs)
Jungrang_sentiment['tokenized'] = Jungrang_sentiment['tokenized'].apply(lambda x: [item for item in x if item not in stop_words])

C:\Users\user\AppData\Local\Temp\ipykernel_34032\1288747233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Jungrang_sentiment['tokenized'] = Jungrang_sentiment['content'].apply(tokenizer.morphs)
C:\Users\user\AppData\Local\Temp\ipykernel_34032\1288747233.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Jungrang_sentiment['tokenized'] = Jungrang_sentiment['tokenized'].apply(lambda x: [item for item in x if item not in stop_words])


## Split

In [9]:
train_data, test_data, train_labels, test_labels = train_test_split(
    Jungrang_sentiment['content'], Jungrang_sentiment['sentiment'], test_size=0.2, random_state=42)

## Vectorizer

In [10]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
train_data_tfidf = tfidf_vectorizer.fit_transform(train_data)
test_data_tfidf = tfidf_vectorizer.transform(test_data)

## MultinomialNB

### 학습

In [11]:
classifier = MultinomialNB() # 나이브 베이즈(Naive Bayes) 분류 모델, 주로 텍스트 분류 문제에 사용
classifier.fit(train_data_tfidf, train_labels)

MultinomialNB()

In [12]:
predictions = classifier.predict(test_data_tfidf)

In [13]:
accuracy = accuracy_score(test_labels, predictions)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.91


In [14]:
print(classification_report(test_labels, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       113
           1       0.91      1.00      0.95      1081

    accuracy                           0.91      1194
   macro avg       0.45      0.50      0.48      1194
weighted avg       0.82      0.91      0.86      1194



c:\Users\user\Desktop\saltlux_project\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\Desktop\saltlux_project\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\Desktop\saltlux_project\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

### 예측

In [16]:
# 데이터 로드
Yeongdeungpo_reviews = pd.read_csv('../preprocess/naver_map_crawling_data_영등포구 병원_1reviews.csv')
Yeongdeungpo_reviews.head(3)

,name,category,review,date,revisit
0,건양의료재단 김안과병원,안과,보험수가,방문일\n11.25.토\n2023년 11월 25일 토요일,1번째 방문
1,건양의료재단 김안과병원,안과,...,방문일\n11.20.월\n2023년 11월 20일 월요일,6번째 방문
2,건양의료재단 김안과병원,안과,좋아요,방문일\n11.15.수\n2023년 11월 15일 수요일,1번째 방문


In [17]:
Yeongdeungpo_reviews = Yeongdeungpo_reviews[['name', 'review']]

# review null값 삭제
print(Yeongdeungpo_reviews['review'].isnull().sum())
Yeongdeungpo_reviews.dropna(inplace=True)
Yeongdeungpo_reviews.info()

51
<class 'pandas.core.frame.DataFrame'>
Index: 16288 entries, 0 to 16338
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    16288 non-null  object
 1   review  16288 non-null  object
dtypes: object(2)
memory usage: 381.8+ KB


In [18]:
Yeongdeungpo_reviews = Yeongdeungpo_reviews.reset_index(drop=True)
Yeongdeungpo_reviews.head(2)

,name,review
0,건양의료재단 김안과병원,보험수가
1,건양의료재단 김안과병원,...


In [19]:
Yeongdeungpo_reviews['review'] = Yeongdeungpo_reviews['review'].str.replace('\n', ' ')
Yeongdeungpo_reviews['review'] = Yeongdeungpo_reviews['review'].str.replace('[~!]', ' ', regex=True) # regex=True : 정규표현식으로 해석(디폴트)
# Jungrang_sentiment['content'] = Jungrang_sentiment['content'].str.replace('[^가-힣]', ' ')
Yeongdeungpo_reviews['review'] = Yeongdeungpo_reviews['review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백 제외하고 모두 제거
Yeongdeungpo_reviews['review'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
print(Yeongdeungpo_reviews.isnull().sum())

name      0
review    0
dtype: int64


In [20]:
Yeongdeungpo_reviews = Yeongdeungpo_reviews.dropna(how='any') # Null 값 제거
print('전처리 후 샘플의 개수 :',len(Yeongdeungpo_reviews))

전처리 후 샘플의 개수 : 16288


In [21]:
stop_words = ['은', '는', '이', '가', '을', '를', '다', '의', '가', '이', '은', '한', '에', '하', '고', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
Yeongdeungpo_reviews['tokenized'] = Yeongdeungpo_reviews['review'].apply(tokenizer.morphs)
Yeongdeungpo_reviews['tokenized'] = Yeongdeungpo_reviews['tokenized'].apply(lambda x: [item for item in x if item not in stop_words])

In [22]:
# 새로운 데이터에 대한 TF-IDF 벡터화
new_data_tfidf = tfidf_vectorizer.transform(Yeongdeungpo_reviews['review'])

# 학습된 모델을 사용하여 예측
new_predictions = classifier.predict(new_data_tfidf)

# 새로운 리뷰에 대한 감성 예측 결과를 DataFrame으로 변환
result_df = pd.DataFrame({'리뷰': Yeongdeungpo_reviews['review'], '예측 감성': new_predictions})

# CSV 파일로 저장
result_df.to_csv('../results/ML/MultinomialNB_pred_results.csv', index=False, encoding='utf-8')

print("예측 결과가 'MultinomialNB_pred_results.csv' 파일로 저장되었습니다.")

예측 결과가 'MultinomialNB_pred_results.csv' 파일로 저장되었습니다.


In [24]:
df = pd.read_csv('../results/ML/MultinomialNB_pred_results.csv')
df[df['예측 감성'] == 0]

,리뷰,예측 감성
1772,대기시간이 길어여 ㅠㅠ,0
3238,나이드신 직원분들은 친절하신데 젊은 간호사들은 불친절,0
7908,시설은 좋으나 그냥 쏘쏘,0
8670,간호사쌤들은 친절도가 케바케임다 나 다니는데는 친절하신데 옆 데스크는 불친절...,0
12008,의사선생님은 친절하신데 카운터실장?은 불친절,0
12073,"카운터 불친절, 불쾌",0
14636,불친절,0
14856,안내디스크 직원 불친절..기분 더러워서 다신 못갈 곳,0
16162,가끔 불친절,0


## Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression

# 모델 생성
classifier = LogisticRegression()

# 모델 학습
classifier.fit(train_data_tfidf, train_labels)

LogisticRegression()

In [26]:
predictions = classifier.predict(test_data_tfidf)
accuracy = accuracy_score(test_labels, predictions)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.91


In [27]:
print(classification_report(test_labels, predictions))

              precision    recall  f1-score   support

           0       1.00      0.04      0.08       113
           1       0.91      1.00      0.95      1081

    accuracy                           0.91      1194
   macro avg       0.95      0.52      0.52      1194
weighted avg       0.92      0.91      0.87      1194



## Support Vector Machine

In [30]:
from sklearn.svm import SVC

# 모델 생성
classifier = SVC()

# 모델 학습
classifier.fit(train_data_tfidf, train_labels)

SVC()

In [31]:
predictions = classifier.predict(test_data_tfidf)
accuracy = accuracy_score(test_labels, predictions)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.91


## Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier

# 모델 생성
classifier = RandomForestClassifier()

# 모델 학습
classifier.fit(train_data_tfidf, train_labels)

RandomForestClassifier()

In [33]:
predictions = classifier.predict(test_data_tfidf)
accuracy = accuracy_score(test_labels, predictions)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.91
